In [ ]:
# History
# Name      Date          Description                   Issue
# Bofan     Nov 20        upload and load dataset.      file too large to load and causing crash
# Emanuel   Dec 1         removing reviews stopwords    

# 1. Preprocessing  dataset

In [ ]:
# Preprocess data: removing irrelevant features
# Includes: tokenization, case-folding, stopword removal, stem words, weight words
# Includes: tokenization, case-folding, stop word removal
# Return a ranked list of reviews 

# dataset download link: https://jmcauley.ucsd.edu/data/amazon_v2/index.html
# example name is sport and oudoor

## Bofan      load dataset

In [138]:
from os import path
import os
import pandas as pd
import numpy as np
import pandas as pd
import missingno as msno

import matplotlib.pyplot as plt
import glob



#### read df

In [128]:
################## 
'''
Create sample data and chunks
'''
##################


# generate sample data

#filename = location+path
#samplepath = location+sample

# for i in chunks:
#   df = i
#   break
# # save to sample.csv in data folder
# df.to_csv(samplepath)



# generate a smaller chunk dataset for futrue use
import pandas as pd


def sampledata(filename,chunkpath, chunksize):
  chunks = pd.read_json(filename, lines=True, chunksize = chunksize)
  i=0
  for c in chunks:
      #print(c)
      i += 1
      print(i,"th of",str(chunksize))
      c.to_csv(chunkpath+str(i)+'.csv')

################## 
'''
read chunks and write back
'''
##################

def printdfsize(df):  # get data frame size by compute row * col
    row, col = df.shape
    print('df size', row * col)


def cleanchunk(chunkfilepath):
    # chunkfilepath = 'data/chunkdata/chunk1.csv'

    # read and load sample.csv data
    df = pd.read_csv(chunkfilepath, low_memory=True)
    print('#'*20,  'dataset load in shape')
    print(df.shape)

    printdfsize(df)

    # print(df.head())
    print( '#'*20, 'column names')
    print(list(df.columns))
    # msno.matrix(df)

    df = df.drop(columns=['Unnamed: 0','vote', 'image','style','reviewTime','reviewerName','Unnamed: 0', 'reviewerID'])
    df = df.dropna()
    print('#'* 20, 'shape after first drop')
    print(df.shape)
    printdfsize(df)

    print('#'* 20, 'shape after drop false verified')
    df = df[df.verified == True]
    df = df.drop(columns=['verified'])
    print(df.shape)
    printdfsize(df)
    # print(list(df.columns))
    # df.head()
    # df['overall'].value_counts()

    # remove rows that has overall rate lower than 4 
    ratestd = 4
    df = df[df.overall > ratestd-1]
    print('#'* 20, 'df shape after filtering rate lower than',str(ratestd),'is')
    print( df.shape)
    printdfsize(df)
    df.head()

    # df

    df.to_csv(chunkfilepath) 


def writenewchunk(list_of_files):  # after clean data over write chunk.csv 
    for i in (list_of_files):
        print('-'*100)
        print("clean chunk ",str(i))
        print('-'*100)
        chunkfilepath = 'data/chunkdata/'+i
        cleanchunk(chunkfilepath)


################## 
'''
mergy chunks into one df
'''
##################


def chunkstoone():
    # setting the path for joining multiple files
    files = os.path.join("data/chunkdata/", "chunk*.csv")

    # list of merged files returned
    files = glob.glob(files)

    print("Resultant CSV after joining all CSV files at a particular location...");

    # joining files with concat and read_csv
    df = pd.concat(map(pd.read_csv, files), ignore_index=False)
    df = df.drop(columns=['Unnamed: 0'])
    print( df.shape)
    printdfsize(df)
    return df


################## 
'''
asin
'''
##################

def checkasinreviewcount(df):   # df['asin'].value_counts()
    df['asin'].value_counts().plot()
    plt.title('product\'s review count')
    plt.ylabel('number of review')
    plt.xlabel('product\'s ASIN')


def createasinlist(numbersofreviews, df): # create condition list of product that has threshold number of reviews and plot
    asincount = list()
    for i in range(numbersofreviews):
        asin = df['asin'].value_counts()
        asin = asin.to_frame()
        asinlist= asin.index[asin['asin'] >i].tolist()
        a = len(asinlist)
        asincount.append(a)
    # plt.plot(asincount)
    # plt.title('number of product for certain number of review')
    # plt.ylabel('number of product')
    # plt.xlabel('number of review per product (review/product)')
    return asinlist


In [ ]:
def main():

    # B
    # dataset web address https://jmcauley.ucsd.edu/data/amazon_v2/categoryFiles/Sports_and_Outdoors.json.gz

    location = 'data/'
    chunklocation = 'chunkdata/chunk'
    path = 'Sports_and_Outdoors.json'
    #pathgz = 'Sports_and_Outdoors.json.gz' 
    # sample = 'sample'
    # file location and name

    #filenamegz = location+pathgz
    filename = location + path
    chunkpath = location + chunklocation

    print(filename)
    print(chunkpath)

    # os.mkdir('data/chunkdata')

    print(chunkpath+str(0)+'.csv')

    #####################################
    # create chunks from original dataset
    sampledata(filename,chunkpath, chunksize = 10**5)


    # drop columns and write backto chunks
    list_of_files = os.listdir('data/chunkdata/')
    # print(list_of_files)
    writenewchunk(list_of_files)


    # read chunks in one data frame
    df = chunkstoone()

    # select rows of data that asin has review at least of review number.
    # then write is as prep.csv for ML session
    reviewnumber = 200
    asinlist = createasinlist(reviewnumber,df)

    df = df[df['asin'].isin(asinlist)]
    print('#'* 20, 'df shape after filtering product review number lower than',str(reviewnumber),'is')
    print( df.shape)
    printdfsize(df)
    df.to_csv('data/prep.csv')

### E